<a href="https://colab.research.google.com/github/sanakhanbano/api/blob/master/twitter_bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#BI LSTM CODE 
import pandas as pd 
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding
#from tensorflow.keras.embeddings import Embedding
#from tensorflow.keras.initializers import Contant
from tensorflow.keras import backend

In [1]:
embedding_path = "/home/skb/glove.twitter.27B.100d.txt" ## change 
# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(embedding_path, "r")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0] # the first word is the key
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")
w2v = get_word2vec(embedding_path)

FileNotFoundError: ignored

In [0]:
#Preprocessing the text:

#df_rage = pd.read_csv(os.path.join(dir_path,'processed_rage.csv'))
#df_happy =  pd.read_csv(os.path.join(dir_path,'processed_happy.csv'))
#df_gore =  pd.read_csv(os.path.join(dir_path,'processed_gore.csv'))
#df_creepy =  pd.read_csv(os.path.join(dir_path,'processed_creepy.csv'))
# create a random balances dataset of all of the categories
#length = np.min([len(df_rage),len(df_happy),len(df_creepy),len(df_gore)])
#df_final = pd.concat([df_rage[:length], df_happy[:length], df_gore[:length], df_creepy[:length]], ignore_index=True)

In [0]:
df = pd.DataFrame()
df_final = pd.DataFrame()
df_final= df = pd .read_csv('comment_final 1.csv',encoding='utf-8',nrows=10)
df_final.head()

,LABEL,USER,COMMENTS
0,0,Ramyunplz95,"['Haha I hear ya! Maybe a really nice gentle, ..."
1,0,payinghomage322,"['How is it gritty though?', '> Pixelmon\n\nHo..."
2,1,somniaprophitiae,['Can someone please tell me what... are the s...
3,0,pigwidgeon__,['A lot of presidents received animals as gift...
4,1,Aimilia97,"[""That's true I guess I just try to have some ..."


In [0]:
#Tokenizing :

import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet as wn
tknzr = TweetTokenizer()
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
token_list = (df_final['COMMENTS'].apply(get_tokens))


[nltk_data] Downloading package wordnet to /home/skb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/skb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#Baseline models
# the object is a word2vec dictionary with value as array vector,
# creates a mean of word vector for sentences


In [0]:
#Output variable:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_new = df_final['LABEL']
Y_new = le.fit_transform(Y_new)

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(token_list)
vocab_size = len(t.word_index) + 1

In [0]:
#Preparing the input variable
# integer encode the documents
max_len=400
encoded_docs = t.texts_to_sequences(token_list)
# pad documents to a max length of 4 words
max_length = max_len
X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
Y = Y_new

In [0]:
#plitting the data to training and testing
## now splitting into test and training data
from sklearn.model_selection import train_test_split
#X_train,X_test, Y_train, Y_test =  train_test_split(X, y,test_size =0.20,random_state= 4 )
X_train,X_test, Y_train, Y_test =  train_test_split(X, Y,test_size =0.15)

In [0]:
#III. Bidirectional LSTM:

# get the embedding matrix from the embedding layer
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
 embedding_vector = w2v.get(word)
 if embedding_vector is not None:
  embedding_matrix[i] = embedding_vector

In [0]:
tf.config.experimental.set_memory_growth(gpu, True)

NameError: name 'gpu' is not defined

In [0]:
TF_FORCE_GPU_ALLOW_GROWTH=1

In [0]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [0]:
#Final Model
import tensorflow.keras
import tensorflow  as tf
# main model
input = tf.keras.layers.Input(shape=(max_len,))
model = tf.keras.layers.Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_len)(input)
model = tf.keras.layers.Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
#model = Bidirectional(LSTM(64))(model)
model = tf.keras.layers.TimeDistributed(Dense(100,activation='relu'))(model)
model = tf.keras.layers.Flatten()(model)
model = tf.keras.layers.Dense(100,activation='relu')(model)
output = tf.keras.layers.Dense(3,activation='softmax')(model)
model = tf.keras.Model(input,output)


In [0]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.01),metrics=['accuracy']) 

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 400, 100)          714600    
_________________________________________________________________
bidirectional (Bidirectional (None, 400, 200)          160800    
_________________________________________________________________
time_distributed (TimeDistri (None, 400, 100)          20100     
_________________________________________________________________
flatten (Flatten)            (None, 40000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               4000100   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303   

In [0]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [0]:
#Fit the training data to the model:
model.fit(X_train,Y_train,validation_split=0.25, epochs = 20, verbose = 2)

Train on 6 samples, validate on 2 samples
Epoch 1/20
6/6 - 0s


CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node Adam/Adam/update/AssignSubVariableOp/_33}}]]
	 [[Reshape_11/_30]] [Op:__inference_distributed_function_12833]

Function call stack:
distributed_function


In [0]:
#IV: RESULTS

# evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))
#Accuracy: 74.593496

In [0]:
#Classification report
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')


In [0]:
#References:
#[1]: Lample, Guillaume, Miguel Ballesteros, Sandeep Subramanian, Kazuya Kawakami, and Chris Dyer. “Neural architectures for named entity recognition.” arXiv preprint arXiv:1603.01360 (2016).
#[2] Duong, Chi Thang, Remi Lebret, and Karl Aberer. “Multimodal Classification for Analysing Social Media.” The 27th European Conference on Machine Learning and Principles and Practice of Knowledge Discovery in Databases (ECML-PKDD), 2017